## 제주시 일반음식점 현황
- x,y 좌표 모두 EPSG:2097 좌표계

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd

In [2]:

일반음식점 =  pd.read_csv("행정안전부 일반음식점.csv",low_memory=False,encoding="cp949")

In [4]:
일반음식점.columns

Index(['번호', '개방서비스명', '개방서비스id', '개방자치단체코드', '관리번호', '인허가일자', '인허가취소일자',
       '영업상태구분코드', '영업상태명', '상세영업상태코드', '상세영업상태명', '폐업일자', '휴업시작일자', '휴업종료일자',
       '재개업일자', '소재지전화', '소재지면적', '소재지우편번호', '소재지전체주소', '도로명전체주소', '도로명우편번호',
       '사업장명', '최종수정시점', '데이터갱신구분', '데이터갱신일자', '업태구분명', '좌표정보(x)', '좌표정보(y)',
       '위생업태명', '남성종사자수', '여성종사자수', '영업장주변구분명', '등급구분명', '급수시설구분명', '총종업원수',
       '본사종업원수', '공장사무직종업원수', '공장판매직종업원수', '공장생산직종업원수', '건물소유구분명', '보증액',
       '월세액', '다중이용업소여부', '시설총규모', '전통업소지정번호', '전통업소주된음식', '홈페이지',
       'Unnamed: 47'],
      dtype='object')

In [5]:
일반음식점_2 = 일반음식점.drop(["번호","개방서비스명","개방서비스id","관리번호","보증액","건물소유구분명",
                      '본사종업원수', '공장사무직종업원수', '공장판매직종업원수', '공장생산직종업원수',
                      '최종수정시점', '데이터갱신구분', '데이터갱신일자','재개업일자', '소재지전화',
                       "인허가일자","홈페이지","월세액","전통업소주된음식",'Unnamed: 47','인허가일자', '인허가취소일자',
                      "영업상태구분코드",'폐업일자', '휴업시작일자', '휴업종료일자','소재지우편번호',
                      "전통업소지정번호","다중이용업소여부"],axis=1)

In [7]:
일반음식점_2["시도"] = 일반음식점_2["개방자치단체코드"].apply(lambda x : str(x)[:2])

In [8]:
제주_일반음식점 = 일반음식점_2[일반음식점_2["시도"] == "65"]
제주_일반음식점 = 제주_일반음식점[제주_일반음식점["영업상태명"] == "영업/정상"].reset_index(drop=True)

In [9]:
제주_일반음식점.to_csv("제주도 일반음식점 현황.csv",encoding="utf-8-sig",index=False)

In [18]:
제주_일반음식점_re = pd.read_csv("제주도 일반음식점 현황.csv")

In [19]:
cnt = 0
while cnt < len(제주_일반음식점_re):
    if str(제주_일반음식점_re.loc[cnt,"소재지전체주소"]) == "nan":
        제주_일반음식점_re.loc[cnt,"주소"] = 제주_일반음식점_re.loc[cnt,"도로명전체주소"]
    else:
        제주_일반음식점_re.loc[cnt,"주소"] = 제주_일반음식점_re.loc[cnt,"소재지전체주소"]
    cnt+=1

In [20]:
제주_일반음식점_re.drop(["소재지전체주소","도로명전체주소",
                  "업태구분명","도로명우편번호","소재지면적"],axis=1,inplace=True)

In [28]:
제주_일반음식점_re["법정동"] = 제주_일반음식점_re["주소"].apply(lambda x : x.split(" ")[2])
제주_일반음식점_re["시도"] = 제주_일반음식점_re["주소"].apply(lambda x : x.split(" ")[1])
#제주_일반음식점_re["법정동"] = 제주_일반음식점_re["법정동"].apply(lambda x : " ".join(x))

In [29]:
제주시_일반음식점_re = 제주_일반음식점_re[제주_일반음식점_re["시도"] == "제주시"]
서귀포시_일반음식점_re = 제주_일반음식점_re[제주_일반음식점_re["시도"] == "서귀포시"]

좌표계 변환하기
- 너무 오래걸림 ㅡㅡ

In [31]:
from pyproj import Proj, transform
def get_좌표계변환(data,위도_col,경도_col):
    # 중부원점(Bessel): 서울 등 중부지역 EPSG:2097
    proj_1 = Proj(init='epsg:2097')
    # WGS84 경위도: GPS가 사용하는 좌표계 EPSG:4326
    proj_2 = Proj(init='epsg:4326')
    위경도 = data[[경도_col,위도_col]]
    위도 = [] ; 경도 = []
    
    DataFrame = 위경도.copy()
    for idx, row in DataFrame.iterrows():
        x, y = row[경도_col], row[위도_col]
        x_, y_ = transform(proj_1, proj_2, x, y)
        경도.append(x_)
        위도.append(y_)
    data["위도"] = 위도  ; data["경도"] = 경도
    return data


In [32]:
서귀포시_일반음식점_위경도 = get_좌표계변환(서귀포시_일반음식점_re,"좌표정보(y)","좌표정보(x)")

In [34]:
#서귀포시_일반음식점_위경도.to_csv("서귀포시 일반음식점 현황.csv",index=False,encoding="utf-8-sig")

In [35]:
제주시_일반음식점_위경도 = get_좌표계변환(제주시_일반음식점_re,"좌표정보(y)","좌표정보(x)")

In [37]:
제주_일반음식점_위경도 = pd.concat([서귀포시_일반음식점_위경도,제주시_일반음식점_위경도],ignore_index=True,axis=0)
제주_일반음식점_위경도.head(3)

In [40]:
제주_일반음식점_위경도.to_csv("제주도 일반음식점 현황 위경도.csv",encoding="utf-8-sig",index=False)

In [15]:
DataFrame = 위경도.copy()
for idx, row in DataFrame.iterrows():
    x, y = row['좌표정보(x)'], row['좌표정보(y)']
    x_, y_ = transform(proj_1, proj_2, x, y)
    경도.append(x_)
    위도.append(y_)

In [ ]:
DataFrame = 위경도.copy()
cnt = 0
while cnt < len(DataFrame):
    x,y = DataFrame.iloc[cnt,0], DataFrame.iloc[cnt,1]
    x_,y_ = transform(proj_1, proj_2, x, y)
    경도.append(x_)
    위도.append(y_)
    cnt+=1

In [25]:
len(경도)

51